In [1]:
import collections
import copy
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
with open(r'AnnaKarenina.txt', "r") as file:
    anna_karenina = file.read().lower().replace("\n", " ")

with open("./WarAndPeace.txt", "r") as file:
    war_and_peace_ru = file.read().lower().replace("\n", " ")
    
with open("./WarAndPeaceEng.txt", "r") as file:
    war_and_peace_en = file.read().lower().replace("\n", " ")

In [3]:
a = ord('а')
rus_alphabet = ''.join([chr(i) for i in range(a, a+32)])

In [4]:
a1 = ord('a')
en_alphabet = ''.join([chr(i) for i in range(a1, a1+26)])

In [5]:
rus_alphabet = rus_alphabet + ' '
en_alphabet = en_alphabet + ' '

План на первый этап
1. Отфильтровать текст, оставив только буквы и пробелы.
2. Посчитать частоты.
3. Зашифровать перестановкой 
4. Расшифровать


In [17]:
%load_ext autoreload
%autoreload 2
from assignment3_modules import filter_text, get_freqs, unigram_encode_decode, \
                                get_encode_mapping, get_decode_mapping, similar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
test_text = """
В ОВИРе эта сука мне и говорит:
– Каждому отъезжающему полагается три чемодана. Такова установленная норма. Есть специальное распоряжение министерства.
Возражать не имело смысла. Но я, конечно, возразил:
– Всего три чемодана?! Как же быть с вещами?
– Например?
– Например, с моей коллекцией гоночных автомобилей?
– Продайте, – не вникая, откликнулась чиновница.
Затем добавила, слегка нахмурив брови:
– Если вы чем-то недовольны, пишите заявление.
– Я доволен, – говорю.
После тюрьмы я был всем доволен.
– Ну, так и ведите себя поскромнее…
Через неделю я уже складывал вещи. И, как выяснилось, мне хватило одного-единственного чемодана.
Я чуть не зарыдал от жалости к себе. Ведь мне тридцать шесть лет. Восемнадцать из них я работаю. Что-то зарабатываю, покупаю. Владею, как мне представлялось, некоторой собственностью. И в результате – один чемодан. Причем довольно скромного размера. Выходит, я нищий? Как же это получилось?!
"""

In [19]:
train_texts = filter_text(anna_karenina + war_and_peace_ru+war_and_peace_en, rus_alphabet)
unigram_freqs = get_freqs(train_texts)
encode_mapping = get_encode_mapping(unigram_freqs)
filtered_text = filter_text(test_text, rus_alphabet)
encoded_text = unigram_encode_decode(filtered_text, encode_mapping)

In [20]:
text_unigram_freqs = get_freqs(encoded_text)
decode_mapping = get_decode_mapping(unigram_freqs, text_unigram_freqs)

decoded_text = unigram_encode_decode(encoded_text, decode_mapping)
print(filtered_text)

в овире эта сука мне и говориткаждому отъезжающему полагается три чемодана такова установленная норма есть специальное распоряжение министерствавозражать не имело смысла но я конечно возразилвсего три чемодана как же быть с вещаминапримернапример с моей коллекцией гоночных автомобилейпродайте не вникая откликнулась чиновницазатем добавила слегка нахмурив бровиесли вы чемто недовольны пишите заявлениея доволен говорюпосле тюрьмы я был всем доволенну так и ведите себя поскромнеечерез неделю я уже складывал вещи и как выяснилось мне хватило одногоединственного чемоданая чуть не зарыдал от жалости к себе ведь мне тридцать шесть лет восемнадцать из них я работаю чтото зарабатываю покупаю владею как мне представлялось некоторой собственностью и в результате  один чемодан причем довольно скромного размера выходит я нищий как же это получилось


In [21]:
print(decoded_text)  # "сука" не перевели(

с осире цта лгда кне и чосоритдаймокг отфеыйажэекг яовачаетлп три ьекомана тадоса глтаносвеннап норка елту ляехиавуное раляорпйение кинилтерлтсасоырайату не икево лкблва но п донеьно соыраыивслечо три ьекомана дад йе збту л сеэакинаярикернаярикер л коеш довведхиеш чоноьнбю астокозивешяромаште не снидап отдвиднгвалу ьиноснихаыатек мозасива лвечда наюкгрис зросиелви сб ьекто немосовунб яищите ыапсвениеп мосовен чосоржяолве тжрукб п збв слек мосовеннг тад и семите лезп яолдрокнееьереы немевж п гйе лдвамбсав сеэи и дад сбплниволу кне юсативо омночоеминлтсенночо ьекоманап ьгту не ыарбмав от йаволти д лезе сему кне тримхату щелту вет солекнамхату иы нию п разотаж ьтото ыаразатбсаж яодгяаж свамеж дад кне яремлтасвпволу недоторош лозлтсеннолтуж и с реыгвутате  омин ьекоман яриьек мосовуно лдрокночо раыкера сбюомит п ниэиш дад йе цто яовгьиволу


In [22]:
similar(filtered_text, decoded_text)

0.009445100354191263

Некоторые слова, типа частиц угадываются или близко переведены, мне -> кне, но -> но, и -> и, но по большей части ничего не понятно.
Перейдем к биграммам. План следующий.
1. Посчитать частоты пар букв
2. Повторить тестирование

In [37]:
def get_bigram_freqs(text):
    freqs = collections.Counter([text[i:i+2] for i in range(len(text) - 2)])
    freqs = {k: v / sum(freqs.values()) for k, v in freqs.items()}
    freqs = dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))
    return freqs

def bigram_decode(text, mapping):
    decoded_text = ['pad'] * len(text) 
    for text_bi, orig_bi in mapping.items(): # сначала заполняем самые частотные
        index = text.find(text_bi)
        while index != -1:
            if decoded_text[index] == 'pad':
                decoded_text[index] = orig_bi[0]
            if decoded_text[index + 1] == 'pad':
                decoded_text[index + 1] = orig_bi[1] 
            index = text.find(text_bi, index + 1)
    return ''.join(decoded_text)

In [24]:
bigram_freqs = get_bigram_freqs(train_texts)
text_bigram_freqs = get_bigram_freqs(encoded_text)
decode_mapping = get_decode_mapping(bigram_freqs, text_bigram_freqs)

decoded_text = bigram_decode(encoded_text, decode_mapping)

In [25]:
decoded_text

'бме еа  эк  ср  о а  наое тне  ктроигм  кинеогжя гнил ркливы нснналя ноог окаье  оу иаи  ь аго ни ех ои и  саттретди   рамотейвлко  отко илк ин стурае п  а  бдает свы п  и о ннла ли о тура ера еотоснналя ноог о кьрв  боп  со  ематгоонедскгоонедск свочсаенл ь жтаказови ейиеови м чееь сомноозо  а   ко к н  ивпеняпрм плти  кооюч дя  роевиеп  сь д   вош гнебыбле ак зно  аля  т а ре т до нивако  чатть кок нре ть аеое теноонь  свцев т нбосо  я  ре ть аесокаь но  ико  сее нионнле а ый кнн а ирува нув  снпреин то  енана кьо  ракоеон п а  хн деетонои овлита ивлаи отоля ноого нлмп  а  чтяхлотм   ееео инан сее    и п а  снеио п пхи и пь мо тня гоео п пбо вохт нрааукавал у точтраб дин вниткувово превве кьв а  имли иат ееон  а иу мееа сче ивлаи  иов но ыанн одкад  мнотааля нооанинейя  ре т ди  снле и отора дср о  енок т нводуае кьрв  э тоил шбееоно'

In [26]:
similar(decoded_text, filtered_text)

0.018890200708382526

Метрика немного выросла, но похоже, что в пределах стат. погрешности, так как качество упало и даже частицы не угадываются.

Зная вероятности встретить биграмму в тренировочном тексте можем путем перестановок в декодированном считать максимальное правдоподобие для всех биграмм. Новое шифрование оставляем, если увеличилось правдоподобие.

In [38]:
from assignment3_modules import accept

In [39]:
class MCMC:
    def __init__(self, known_freqs, alphabet, n_iter=100000, verbose=5000):
        self.n_iter = n_iter
        self.verbose = verbose
        self.known_freqs = known_freqs
        self.alphabet = alphabet 
        self.nan_replacement = 1 / len(rus_alphabet) ** 2
        
    def _mutate(self, text):
        letters = np.random.choice(self.alphabet, 2, replace=False)
        for i in range(len(text)):
            if text[i] == letters[0]:
                text[i] = letters[1]
            elif text[i] == letters[1]:
                text[i] = letters[0]
        return text
        
    def loglikelihood(self, text):
        bigram_counts = collections.Counter([''.join(text[i: i + 2]) for i in range(len(text) - 2)])
        return np.sum([count * np.log(self.known_freqs.get(bigram, self.nan_replacement)) 
                       for bigram, count in bigram_counts.items()])
        
    def decode(self, text):
        best_decoded_text = copy.copy(text)
        best = self.loglikelihood(text)
        current = self.loglikelihood(text) 
        for iteration in range(self.n_iter):
            new_text = self._mutate(copy.copy(text))
            new = self.loglikelihood(new_text)
            if accept(current, new):
                text = new_text
                current = new
                if current > best:
                    best = current
                    best_decoded_text = copy.copy(text)
            if iteration % self.verbose == 0:
                res = ''.join(best_decoded_text)
                print(res)
                print('similarity {}'.format(similar(res, filtered_text)))
                print('############################################################')
        return ''.join(best_decoded_text)

In [36]:
mcmc = MCMC(bigram_freqs, list(rus_alphabet))

mcmc.decode(list(encoded_text))

жсмжрчхспъфс экфсцшхсрсбмжмчръкфщьмцэсмъохтщфидхцэснмафбфхъ усъчрсяхцмьфшфсъфкмжфсэ ъфшмжахшшфусшмчцфсх ъес нхврфаешмхсчф нмчущхшрхсцршр ъхч ъжфжмтчфщфъесшхсрцхамс цг афсшмсускмшхяшмсжмтчфтраж хбмсъчрсяхцмьфшфскфксщхслгъес сжхдфцршфнчрцхчшфнчрцхчс сцмхйскмаахкврхйсбмшмяшгысфжъмцмлрахйнчмьфйъхсшхсжшркфусмъкаркшэаф есяршмжшрвфтфъхцсьмлфжрафс ахбкфсшфыцэчржслчмжрх арсжгсяхцъмсшхьмжмаешгснрзръхстфужахшрхусьмжмахшсбмжмчинм ахсъичецгсуслгасж хцсьмжмахшшэсъфксрсжхьръхс хлуснм кчмцшххяхчхтсшхьхаисусэщхс кафьгжфасжхдрсрскфксжгу шрам есцшхсыжфърамсмьшмбмхьрш ъжхшшмбмсяхцмьфшфусяэъесшхстфчгьфасмъсщфам ърскс хлхсжхьесцшхсъчрьвфъесзх ъесахъсжм хцшфьвфъесртсшрысусчфлмъфисяъмъмстфчфлфъгжфиснмкэнфисжафьхискфксцшхснчхь ъфжауам есшхкмъмчмйс мл ъжхшшм ъеисрсжсчхтэаеъфъхссмьршсяхцмьфшснчряхцсьмжмаешмс кчмцшмбмсчфтцхчфсжгымьръсусшрдрйскфксщхспъмснмаэярам е
similarity 0.0059031877213695395
############################################################
с асире фво жуко дне и засаривкоэладу авшегэоьцеду патозое

в овире эта сука мне и говориткаждому отщезжаышему полагается три чемодана такова установленная норма есть спеъиальное распоряжение министерствавозражать не имело смйсла но я конечно возразилвсего три чемодана как же бйть с вешаминапримернапример с моею коллекъиею гоночнйх автомобилеюпродаюте не вникая откликнулась чиновниъазатем добавила слегка нахмурив бровиесли вй чемто недовольнй пиците заявлениея доволен говорыпосле тырьмй я бйл всем доволенну так и ведите себя поскромнеечерез неделы я уже складйвал веши и как вйяснилось мне хватило одногоединственного чемоданая чуть не зарйдал от жалости к себе ведь мне тридъать цесть лет восемнадъать из них я работаы чтото зарабатйваы покупаы владеы как мне представлялось некоторою собственностьы и в результате  один чемодан причем довольно скромного размера вйходит я нишию как же это получилось
similarity 0.0885478158205431
############################################################
в овире эта сука мне и говориткаждому отщезжаышему полагается

'в овире эта сука мне и говориткаждому отщезжаышему полагается три чемодана такова установленная норма есть спеъиальное распоряжение министерствавозражать не имело смйсла но я конечно возразилвсего три чемодана как же бйть с вешаминапримернапример с моею коллекъиею гоночнйх автомобилеюпродаюте не вникая откликнулась чиновниъазатем добавила слегка нахмурив бровиесли вй чемто недовольнй пиците заявлениея доволен говорыпосле тырьмй я бйл всем доволенну так и ведите себя поскромнеечерез неделы я уже складйвал веши и как вйяснилось мне хватило одногоединственного чемоданая чуть не зарйдал от жалости к себе ведь мне тридъать цесть лет восемнадъать из них я работаы чтото зарабатйваы покупаы владеы как мне представлялось некоторою собственностьы и в результате  один чемодан причем довольно скромного размера вйходит я нишию как же это получилось'

Метрика значительно улучшилась, текст стал читаем, "сука" и "чемодан" разгадали. Считаю успехом.
Перейдем к сообщению

In [29]:
to_decode = "დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ"
to_decode_unigram_freqs = get_freqs(to_decode)
to_decode_mapping = get_decode_mapping(unigram_freqs, to_decode_unigram_freqs)
to_decode_text = unigram_encode_decode(to_decode, to_decode_mapping)
mcmc = MCMC(bigram_freqs, list(rus_alphabet), n_iter=80000, verbose=2000)
mcmc.decode(list(to_decode_text))

олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю
similarity 0.016713091922005572
############################################################
екни чх чивите лоржанылхь ини подти лоржанылхь тескт у этомо коогщелия соторхь немсо продитаты ксорее чкемо чх чке квенани прачиныло и понудите жаскижанылхь ганн за покневлее детчертое завалие сурка ботя соледло я лидемо ле огещай
similarity 0.02042711234911792
############################################################
есни дй дилите ровзанырйь ини помти ровзанырйь текст у этого соожшерия котовйь негко пвомитаты сковее дсего дй дсе сленани пвадиныро и понумите заксизанырйь жанн ча поснелрее метдевтое чаларие кувса ботя коремро я римего ре ожешаф
similarity 0.014856081708449397
############################################################
если вы видите норчальный или

если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаф
similarity 0.014856081708449397
############################################################
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаф
similarity 0.014856081708449397
############################################################
если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаф
similarity 0.014856081708449397
############################################################
если вы вимите нордальный ил

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаф'

Идея сообщения понятна. Было бы здорово получить максимальный жалл за четвертое задание из трех существующих в курсе)